In [429]:
#轉換資料所需暫存變數
playState = 0 #0:遊戲初始化階段 1:打牌階段 2:和牌 3:流局
startPlayer = 0
eventState = 0 #0:nothing 1:REACH 2:N 3:DORA 4:INITDORA
eventPlayer = -1
windlist = [0,1,2,3]#0:東 1:南 2:西 3:北 #
score = [250,250,250,250]#
dora = [-1,-1,-1,-1,-1] #
hai0 = []#
hai1 = []#
hai2 = []#
hai3 = []#
N0 = []#
N1 = []#
N2 = []#
N3 = []#
rounds = 0#
play1Dis = []
play2Dis = []
play3Dis = []
play4Dis = []
IfREACHPlayer = [0,0,0,0] #
datalist = []
rowData = [0]*183
#注意 剛加入還是可以透過rowData改datalist的值

In [430]:
#函數宣告集中處
def GameResetValue():
    global playState,windlist,score,dora,hai0,hai1,hai2,hai3,N0,N1,N2,N3,rounds,play1Dis,play2Dis,play3Dis,play4Dis,datalist,rowData,IfREACHPlayer,eventPlayer,eventState,startPlayer
    playState = 0
    startPlayer = 0
    eventPlayer = -1
    eventState = 0
    windlist = [1,2,3,4]
    score = [250,250,250,250]
    dora = [-1,-1,-1,-1,-1]
    hai0 = []
    hai1 = []
    hai2 = []
    hai3 = []
    N0 = []
    N1 = []
    N2 = []
    N3 = []
    rounds = 0
    play1Dis = []
    play2Dis = []
    play3Dis = []
    play4Dis = []
    IfREACHPlayer = [0,0,0,0]
    datalist = []
    rowData = [0]*183
    
def intTo16bitBinary(x):#int轉2進制16位
    binaryStr = ''
    for i in range(15,-1,-1):
        bit = (x >> i) & 1
        binaryStr += str(bit)
    return binaryStr

def Binary16bitToInt(str_x,bits):#int轉2進制16位
    x_bits = str_x[:bits]
    decimal_value = int(x_bits, 2)
    return decimal_value

def haitrans(haiX,strlist):#初始化手牌
    haiX.clear()
    hailist = re.split(r'[",]', strlist)
    hailist = [s for s in hailist if s]
    del hailist[0]
    for tile in hailist:
        haiX.append(int(tile))
        
def chow(binary16Bit,eventPlayer):
    #print("chow")
    #print(binary16Bit)
    chowNumber = Binary16bitToInt(binary16Bit,6)
    tileType = int(chowNumber / 21) #0:character 1:circle 2:bamboo
    chowType = int(chowNumber / 3) % 7#0~6種
    chowTile = chowNumber % 3 #0最小 1中間 2最大
    smallTile =  tileType * 9 + chowType #吃的最小牌
    if(eventPlayer == 0):
        N0.append(smallTile)
        N0.append(smallTile + 1)
        N0.append(smallTile + 2)
        hai0.append(smallTile + chowTile)
    elif(eventPlayer == 1):
        N1.append(smallTile)
        N1.append(smallTile + 1)
        N1.append(smallTile + 2)
        hai1.append(smallTile + chowTile)
    elif(eventPlayer == 2):
        N2.append(smallTile)
        N2.append(smallTile + 1)
        N2.append(smallTile + 2)
        hai2.append(smallTile + chowTile)
    elif(eventPlayer == 3):
        N3.append(smallTile)
        N3.append(smallTile + 1)
        N3.append(smallTile + 2)
        hai3.append(smallTile + chowTile)
    
def pung(binary16Bit,eventPlayer):
    #print("pung")
    #print(binary16Bit)
    pungNumber = Binary16bitToInt(binary16Bit,7)
    tileNumber = int(pungNumber / 3)
    if(eventPlayer == 0):
        N0.append(tileNumber)
        N0.append(tileNumber)
        N0.append(tileNumber)
        hai0.append(tileNumber)
    elif(eventPlayer == 1):
        N1.append(tileNumber)
        N1.append(tileNumber)
        N1.append(tileNumber)
        hai1.append(tileNumber)
    elif(eventPlayer == 2):
        N2.append(tileNumber)
        N2.append(tileNumber)
        N2.append(tileNumber)
        hai2.append(tileNumber)
    elif(eventPlayer == 3):
        N3.append(tileNumber)
        N3.append(tileNumber)
        N3.append(tileNumber)
        hai3.append(tileNumber)
    
def kong(binary16Bit,eventPlayer):
    #print("kong")
    #print(binary16Bit)
    if(binary16Bit[11] == '0'): #暗槓
        kongNumber = Binary16bitToInt(binary16Bit,8)
        tileNumber = int(kongNumber / 4)
        if(eventPlayer == 0):
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
        elif(eventPlayer == 1):
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
        elif(eventPlayer == 2):
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
        elif(eventPlayer == 3):
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
    else:#明槓
        kongNumber = Binary16bitToInt(binary16Bit,7)
        tileNumber = int(kongNumber / 3)
        if(eventPlayer == 0):
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
            if(Binary16bitToInt[14] != '0' and Binary16bitToInt[15] != '0'):
                hai0.append(tileNumber)
        elif(eventPlayer == 1):
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
            if(Binary16bitToInt[14] != '0' and Binary16bitToInt[15] != '0'):
                hai1.append(tileNumber)
        elif(eventPlayer == 2):
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
            if(Binary16bitToInt[14] != '0' and Binary16bitToInt[15] != '0'):
                hai2.append(tileNumber)
        elif(eventPlayer == 3):
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
            if(Binary16bitToInt[14] != '0' and Binary16bitToInt[15] != '0'):
                hai3.append(tileNumber)
    
def playMode(element):#進入打牌階段
    global playState,windlist,score,dora,hai0,hai1,hai2,hai3,N0,N1,N2,N3,rounds,play1Dis,play2Dis,play3Dis,play4Dis,datalist,rowData,IfREACHPlayer,eventPlayer,eventState
    #print(element)
    #print(eventState, eventPlayer)
    if(element == 'DORA'):
        eventState = 3
    elif(eventState == 3):
        DORAhai = int(int(element.split('=')[1].strip('"'))/4)
        for i in range(len(dora)):
            if(dora[i] == -1):
                dora[i] = DORAhai
                break
        eventState = 0
    elif(element[0] == 'T'):
        rounds = rounds + 1
        hai0.append(int(element[1:]))
    elif(element[0] == 'D'):
        hai0.remove(int(element[1:]))
        play1Dis.append(int(element[1:]))
    elif(element[0] == 'U'):
        rounds = rounds + 1
        hai1.append(int(element[1:]))
    elif(element[0] == 'E'):
        hai1.remove(int(element[1:]))
        play2Dis.append(int(element[1:]))
    elif(element[0] == 'V'):
        rounds = rounds + 1
        hai2.append(int(element[1:]))
    elif(element[0] == 'F'):
        hai2.remove(int(element[1:]))
        play3Dis.append(int(element[1:]))
    elif(element[0] == 'W'):
        rounds = rounds + 1
        hai3.append(int(element[1:]))
    elif(element[0] == 'G'):
        hai3.remove(int(element[1:]))
        play4Dis.append(int(element[1:]))
    elif(element == 'REACH'):
        eventState = 1
    elif(element == 'N'):
        eventState = 2
    elif(eventState == 1):
        if('who' in element):
            eventPlayer = int(element[5])
            IfREACHPlayer[eventPlayer] = 1
        elif('ten' in element):
            tenlist = re.split(r'[",]', element)
            tenlist = [s for s in tenlist if s]
            del tenlist[0]
            for i in range(len(score)):
                score[i] = int(tenlist[i])
        elif('step' in element and element[6] == '2'):
            eventPlayer = -1
            eventState = 0
    elif (eventState == 2 and eventPlayer != -1):
        Ntype = int(element.split('=')[1].strip('"'))
        binary16Bit = intTo16bitBinary(Ntype)
        if(binary16Bit[12] == '1'):
            pung(binary16Bit,eventPlayer)
        elif(binary16Bit[13] == '1'):
            chow(binary16Bit,eventPlayer)
        elif(binary16Bit[12] == '0' and binary16Bit[13] == '0'):
            kong(binary16Bit,eventPlayer)
        eventState = 0
        eventPlayer = -1
    elif(eventState == 2):
        eventPlayer = int(element[5])
    elif(element == 'AGARI'):
        #print("AGARI")
        playState = 2
    elif(element == 'RYUUKYOKU'):
        playState = 3
    elif(playState == 2 and 'sc' in element):
        sclist = re.split(r'[",]', element)
        scnlist = [s for s in sclist if s]
        del scnlist[0]
        for i in range(len(score)):
            score[i] = int(scnlist[i * 2]) + int(scnlist[i * 2 + 1])
    elif(playState == 3 and 'sc' in element):
        sclist = re.split(r'[",]', element)
        scnlist = [s for s in sclist if s]
        del scnlist[0]
        for i in range(len(score)):
            score[i] = int(scnlist[i * 2]) + int(scnlist[i * 2 + 1])

In [431]:
"""
將牌局資料轉成csv形式
Prevailing wind: 1東 2南 3西 4北
"""
import os
import re
from openpyxl import Workbook, load_workbook

for i in range(2022,2023):
    input_folder = f'E:/專題/data/{i}/scc game'
    wb = load_workbook(f'E:/專題/data/{i}/RewardPredictTrainData{i}.xlsx')
    ws = wb.active
    
    for filename in os.listdir(input_folder):
        txt_path = os.path.join(input_folder, filename)
        with open('E:/專題/data/2022/scc game/scc20220101-4.txt', 'r', encoding='utf-8') as file:
            content = file.read()
            conlist = re.split(r'[<>]', content)
            conlist = [s for s in conlist if s]
            del conlist[1]
            newlist = []
            for item in conlist:
                sub = re.split(r'[ /]', item)
                for ele in sub:
                    newlist.append(ele);
            newlist = [s for s in newlist if s]
            GameResetValue()
            
            for element in newlist:
                
                #初始手牌資訊
                if(element == 'INIT'):
                    GameResetValue()
                    eventState = 4
                elif(eventState == 4):
                    eventState = 0
                    number_str = element.split('=')[1].strip('"')
                    numbers = number_str.split(',')
                    last_value = numbers[-1]
                    DORAhai = int(int(last_value) / 4)
                    for i in range(len(dora)):
                        if(dora[i] == -1):
                            dora[i] = DORAhai
                            break
                elif(playState == 0 and 'oya' in element):
                    startPlayer = int(element[5])
                    for i in range(len(windlist)):
                        windlist[i] = (windlist[i] + 4 - startPlayer) % 4
                elif(playState == 0 and 'ten' in element):
                    tenlist = re.split(r'[",]', element)
                    tenlist = [s for s in tenlist if s]
                    del tenlist[0]
                    for i in range(len(score)):
                        score[i] = int(tenlist[i])
                elif(playState == 1 or playState == 2 or playState == 3):
                    playMode(element)
                elif('hai0' in element):
                    haitrans(hai0, element)
                elif('hai1' in element):
                    haitrans(hai1, element)
                elif('hai2' in element):
                    haitrans(hai2, element)
                elif('hai3' in element and playState == 0):
                    haitrans(hai3, element)
                    playState = 1
        break
    #wb.save(f'E:/專題/data/{i}/RewardPredictTrainData{i}.xlsx')
        